In [1]:
#MultiClass classifier. Predicts one attribute as one of many labels rather than just 2 possible labels.
#This assumes there is only one right answer. We could do multilabel classification and present multiple city solutions to the user?
#https://scikit-learn.org/stable/modules/multiclass.html    a look at different kinds of models

""" method of going through multiple models
 https://medium.com/analytics-vidhya/testing-multiple-machine-learning-models-at-once-without-getting-a-headache-5aefb0e7df03    simple good
 https://github.com/j-planet/machine-learning-big-loop   more advanced, but better. (get parameters.py from this, and the cell for running multiple things"""

"""https://stackoverflow.com/questions/23045318/grid-search-over-multiple-classifiers
Smallest, simple to

https://www.codegrepper.com/code-examples/whatever/gridsearchcv+with+multiple+models
Straightforward

https://www.davidsbatista.net/blog/2018/02/23/model_optimization/
Large, hard to understand, but powerful

https://github.com/davidsbatista/machine-learning-notebooks/blob/master/hyperparameter-across-models.ipynb
Large, hard to understand, but powerful"""


#to do
# find best testing method
# get best parameters

#to ask
#why do his cells not work

# Import necessary modules from repository
from model.data.pipeline import sql_strings as sql, utils as ut
import numpy as np
import pandas as pd


In [6]:
#dataframe of all monthly data
df = ut.load_sql_as_df('SELECT * From MonthlyDataModel;')

#dataframe of month averages to fill in bad values
monthAvg = ut.load_sql_as_df('SELECT * From MonthlyAverages;')



,month,prcpInt,prcpFreq,temp_max_normal,temp_min_normal,snowInt,snowFreq,clouds,dew_point,heat_index,pressure,wind_speed,wind_calm_percentage
0,1,2.533438,47.829146,41.924644,21.303576,-110.709552,-157.315167,55.667977,26.341895,35.417982,1018.967064,8.593669,14.684570
1,2,2.397964,44.998234,45.985372,24.106816,-211.922302,-230.625859,55.013862,28.228661,38.559721,1018.084567,8.813964,13.644477
2,3,2.901085,53.579383,54.495133,31.252366,-309.959310,-297.336583,52.051446,33.052022,45.700434,1016.384440,9.329369,11.746978
3,4,2.869596,53.857123,64.407005,39.586609,-394.670800,-452.524213,49.932187,39.677770,54.337107,1014.836497,9.386922,11.097830
4,5,3.405206,59.647562,73.406937,48.775410,-310.212556,-468.720793,48.941521,48.669454,63.309600,1014.359430,8.659783,12.057727
5,6,3.406922,48.181448,81.395617,56.971941,-57.141252,-142.845293,45.015855,56.405955,71.530007,1013.991145,7.970367,13.445409
6,7,2.762931,5.563629,86.065555,61.435144,-3.008332,-6.212722,40.621356,60.502078,76.008743,1015.030087,7.417123,14.776385
7,8,3.004522,25.909684,85.004020,60.135507,-6.026158,-32.307496,40.020323,59.955251,74.535821,1015.393990,7.050787,16.593875
8,9,2.959741,46.810539,77.926970,52.662073,-156.429374,-337.711363,41.015647,53.890063,67.271141,1015.676289,7.402396,16.429875
9,10,2.877458,47.409751,66.659209,42.108349,-553.695795,-656.829311,45.018233,44.251340,56.376711,1016.886480,7.768246,16.520839


In [7]:
# Getting features and choosing labels. Fill NaN spots with NaN text to manipulate them
labels = df["state"]
features = df.drop(['region', 'latitude', 'longitude', 'elevation', 'region'], axis=1)
features = features.fillna('NaN')

# Changing columns in to months to help compare
featSer = features["month"].to_numpy()
prcpIntM = monthAvg["prcpInt"].to_numpy()
prcpFreqM = monthAvg["prcpFreq"].to_numpy()
temp_max_normalM = monthAvg["temp_max_normal"].to_numpy()
temp_min_normalM = monthAvg["temp_min_normal"].to_numpy()
snow_depthM = monthAvg["snowInt"].to_numpy()
snow_daysM = monthAvg["snowFreq"].to_numpy()
cloudsM = monthAvg["clouds"].to_numpy()
dewM = monthAvg["dew_point"].to_numpy()
heatM = monthAvg["heat_index"].to_numpy()
pressureM = monthAvg["pressure"].to_numpy()
windM = monthAvg["wind_speed"].to_numpy()
windCalmM = monthAvg["wind_calm_percentage"].to_numpy()

prcp_normRay = features["prcp_normal"].to_numpy()
prcp_days_tRay = features["prcp_days_t"].to_numpy()
temp_max_normalRay = features["temp_max_normal"].to_numpy()
temp_min_normalRay = features["temp_min_normal"].to_numpy()
snow_depthRay = features["snow_depth_days"].to_numpy()
snow_daysRay = features["snow_days_t"].to_numpy()
cloudsRay = features["clouds"].to_numpy()
dewRay = features["dew_point"].to_numpy()
heatRay = features["heat_index"].to_numpy()
pressureRay = features["pressure"].to_numpy()
windRay = features["wind_speed"].to_numpy()
windCalmRay = features["wind_calm_percentage"].to_numpy()

#replacing bad values
for i in range(len(featSer)):
    if prcp_normRay[i] == 'NaN' or prcp_normRay[i] == -9999 or prcp_normRay[i] == -8888 or prcp_normRay[i] == -7777 or prcp_normRay[i] == -6666 or prcp_normRay[i] == -5555:
        prcp_normRay[i] = prcpIntM[featSer[i]-1]
    if prcp_days_tRay[i] == 'NaN' or prcp_days_tRay[i] == -9999 or prcp_days_tRay[i] == -8888 or prcp_days_tRay[i] == -7777 or prcp_days_tRay[i] == -6666 or prcp_days_tRay[i] == -5555:
        prcp_days_tRay[i] = prcpFreqM[featSer[i]-1]
    if temp_max_normalRay[i] == 'NaN' or temp_max_normalRay[i] == -9999 or temp_max_normalRay[i] == -8888 or temp_max_normalRay[i] == -7777 or temp_max_normalRay[i] == -6666 or temp_max_normalRay[i] == -5555:
        temp_max_normalRay[i] = temp_max_normalM[featSer[i]-1]
    if temp_min_normalRay[i] == 'NaN' or temp_min_normalRay[i] == -9999 or temp_min_normalRay[i] == -8888 or temp_min_normalRay[i] == -7777 or temp_min_normalRay[i] == -6666 or temp_min_normalRay[i] == -5555:
        temp_min_normalRay[i] = temp_min_normalM[featSer[i]-1]
    if snow_depthRay[i] == 'NaN' or snow_depthRay[i] == -9999 or snow_depthRay[i] == -8888 or snow_depthRay[i] == -7777 or snow_depthRay[i] == -6666 or snow_depthRay[i] == -5555:
        snow_depthRay[i] = snow_depthM[featSer[i]-1]
    if snow_daysRay[i] == 'NaN' or snow_daysRay[i] == -9999 or snow_daysRay[i] == -8888 or snow_daysRay[i] == -7777 or snow_daysRay[i] == -6666 or snow_daysRay[i] == -5555:
        snow_daysRay[i] = snow_daysM[featSer[i]-1]
    if cloudsRay[i] == 'NaN' or cloudsRay[i] == -9999 or cloudsRay[i] == -8888 or cloudsRay[i] == -7777 or cloudsRay[i] == -6666 or cloudsRay[i] == -5555:
        cloudsRay[i] = cloudsM[featSer[i]-1]
    if dewRay[i] == 'NaN' or dewRay[i] == -9999 or dewRay[i] == -8888 or dewRay[i] == -7777 or dewRay[i] == -6666 or dewRay[i] == -5555:
        dewRay[i] = dewM[featSer[i]-1]
    if heatRay[i] == 'NaN' or heatRay[i] == -9999 or heatRay[i] == -8888 or heatRay[i] == -7777 or heatRay[i] == -6666 or heatRay[i] == -5555:
        heatRay[i] = heatM[featSer[i]-1]
    if pressureRay[i] == 'NaN' or pressureRay[i] == -9999 or pressureRay[i] == -8888 or pressureRay[i] == -7777 or pressureRay[i] == -6666 or pressureRay[i] == -5555:
        pressureRay[i] = pressureM[featSer[i]-1]
    if windRay[i] == 'NaN' or windRay[i] == -9999 or windRay[i] == -8888 or windRay[i] == -7777 or windRay[i] == -6666 or windRay[i] == -5555:
        windRay[i] = windM[featSer[i]-1]
    if windCalmRay[i] == 'NaN' or windCalmRay[i] == -9999 or windCalmRay[i] == -8888 or windCalmRay[i] == -7777 or windCalmRay[i] == -6666 or windCalmRay[i] == -5555:
        windCalmRay[i] = windCalmM[featSer[i]-1]


# Recreating filled dataset
features = pd.DataFrame({'prcp_normRay' : prcp_normRay, 'prcp_days_tRay' : prcp_days_tRay, "temp_max_normalRay" : temp_max_normalRay, "temp_min_normalRay" : temp_min_normalRay, "snow_depthRay" : snow_depthRay, "snow_daysRay" : snow_daysRay, "cloudsRay" : cloudsRay, "dewRay" : dewRay, "heatRay" : heatRay, "pressureRay" : pressureRay, "windRay" : windRay, "windCalmRay" : windCalmRay})


In [8]:
features

,prcp_normRay,prcp_days_tRay,temp_max_normalRay,temp_min_normalRay,snow_depthRay,snow_daysRay,cloudsRay,dewRay,heatRay,pressureRay,windRay,windCalmRay
0,5.3,69.0,41.924644,21.303576,0.0,0.0,55.667977,26.341895,35.417982,1018.967064,8.593669,14.68457
1,5.07,65.0,45.985372,24.106816,0.0,0.0,55.013862,28.228661,38.559721,1018.084567,8.813964,13.644477
2,5.8,62.0,54.495133,31.252366,-309.95931,-297.336583,52.051446,33.052022,45.700434,1016.38444,9.329369,11.746978
3,3.77,48.0,64.407005,39.586609,0.0,0.0,49.932187,39.67777,54.337107,1014.836497,9.386922,11.09783
4,3.82,48.0,73.406937,48.77541,0.0,0.0,48.941521,48.669454,63.3096,1014.35943,8.659783,12.057727
...,...,...,...,...,...,...,...,...,...,...,...,...
117031,3.22,25.909684,85.4,62.7,-6.026158,-32.307496,40.020323,59.955251,74.535821,1015.39399,7.050787,16.593875
117032,3.04,46.810539,77.8,53.6,-156.429374,-337.711363,41.015647,53.890063,67.271141,1015.676289,7.402396,16.429875
117033,2.04,47.409751,65.2,40.6,-553.695795,-656.829311,45.018233,44.25134,56.376711,1016.88648,7.768246,16.520839
117034,1.41,49.541899,49.7,28.0,-244.419299,-363.569282,51.45818,35.445112,45.895488,1017.744118,8.266765,15.842452


In [10]:
from sklearn.preprocessing import StandardScaler

#Scalling all data
scaler = StandardScaler()
featPrepped = scaler.fit_transform(features)
featPrepped

array([[ 0.91110925,  0.82029827, -1.25013457, ...,  1.49970583,
         0.37076481,  0.12042121],
       [ 0.82387832,  0.64962544, -1.02606424, ...,  0.99407173,
         0.61304703, -0.24700796],
       [ 1.10074171,  0.52162081, -0.55649706, ...,  0.01996877,
         1.17989276, -0.9173286 ],
       ...,
       [-0.32529439, -0.10091895,  0.03419572, ...,  0.30761668,
        -0.53704064,  0.76911125],
       [-0.56423129, -0.00994405, -0.8210917 , ...,  0.79900818,
         0.01123386,  0.52946061],
       [-0.74627845,  0.01116116, -1.52739358, ...,  1.42744863,
         0.17890146,  0.33213148]])

In [11]:
from sklearn.model_selection import train_test_split

feat_train, feat_test, lab_train, lab_test = train_test_split(featPrepped, labels, random_state=42)
# stratify by city for separating training and testing. stratified sampling

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(feat_train, lab_train)

print(clf.score(feat_test, lab_test))  #This automatically makes predictions on feat_test


0.4152226665299566


In [13]:

#Next 3 cells are simons and do not work

In [14]:
"""import numpy as np
TOP = 5
TMP = 62
CITIES = labels.unique()

predictions_list = []
for i in range(len(df)):
    row = df.iloc[i].sort_values(ascending=False)[:TOP]
    rowDf = row.to_frame().reset_index()
    #print([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()])
    vals = np.insert(np.array([[CITIES[int(x[0])], x[1]] for x in rowDf.to_numpy()]), 0, lab_test.iloc[i])
    predictions_list.append(vals)

cols = ['true']
for i in range(TOP):
    cols.append('city{x}'.format(x=i + 1))
    cols.append('prob{x}'.format(x=i + 1))

preds = pd.DataFrame(predictions_list,
                     columns=cols)
preds
"""

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [ ]:
"""count = 0
for i in range(len(preds)):
    if preds.iloc[i]['true'] == preds.iloc[i]['city1']:
        count += 1
        #print('{i}: {t} and {p}'.format(i=i, t=preds['true'][i], p=preds['city1'][i]))

print('Accuracy: {r}/{t}, {p}%'.format(r=count, t=len(preds), p=round(100*count/len(preds))))"""

In [ ]:
"""CITY = 'Denver'

for i in range(len(preds)):
    row = preds.iloc[i]
    if row['true'] == CITY and row['city1'] != CITY:
        print(row['city1'])
CITIES"""

In [17]:
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

log_reg_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]
dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
kneighbors_params = [{"n_neighbors": 3}, {"n_neighbors": 5}]
naive_bayes_params = [{}]
svc_params = [{"C": 0.01}, {"C": 0.1}, {"C": 1}, {"C": 10}]

modelclasses = [
    ["GradientBoostingClf", GradientBoostingClassifier()],
    ["ExtraTreeClf", ExtraTreesClassifier()],
    ["decision tree", DecisionTreeClassifier()],
    ["random forest", RandomForestClassifier()],
    ["k neighbors", KNeighborsClassifier()],
    ["NearestCentroid", NearestCentroid()],
    #["RadiusNeighbor", RadiusNeighborsClassifier()],
    ["GaussianProcess", GaussianProcessClassifier()],
    ["naive bayes Gaussian NB", GaussianNB()],
    ["naive bayes Bernoulli NB", BernoulliNB()],
    ["support vector machines", SVC()],
    ["Nu support vector machines", NuSVC()],
    ["NN MPL", MLPClassifier()]
]

insights = []
for modelname, model in modelclasses:
    try:
        model.fit(feat_train, lab_train)
        score = model.score(feat_test, lab_test)
        insights.append((modelname, model, score))
    except:
        print(modelname, " did not work \n")


insights.sort(key=lambda x: x[-1], reverse=True)
for modelname, model, score in insights:
    print(modelname, score)





GaussianProcess  did not work 

Nu support vector machines  did not work 

random forest 0.40834672044297093
ExtraTreeClf 0.40028027480602935
k neighbors 0.361349420651468
NN MPL 0.337765321119732
GradientBoostingClf 0.33509929247701403
decision tree 0.3122329698875483
support vector machines 0.29975732303380387
naive bayes Gaussian NB 0.1117339440133985
naive bayes Bernoulli NB 0.07755408961957821
NearestCentroid 0.06360870902689955


c:\users\trenton\documents\programming\ml-team3\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [12]:
from sklearn.model_selection import GridSearchCV

GBR = GradientBoostingRegressor()

para_grid = [{
    "max_depth" : np.linspace(1, 15, 5, dtype = int),
    "n_estimators" : np.linspace(1, 200, 10, dtype = int),
    "learning_rate" :  np.linspace(0.05, 2, 10).round(2)
}]

grid_search = GridSearchCV(GBR, para_grid, cv = 3)
grid_search.fit(X_train, z_train)

print(grid_search.best_params_)